# DUO

In [29]:
import pandas as pd

from visions.core.functional import type_inference_report_frame, compare_detect_inference_frame
from visions.core.implementations import visions_standard_set
from visions.core.implementations.types import *
from visions.core.model.type import evolve_relation
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Addresses

In [30]:
df = pd.read_csv('../data/duo/adressen_instellingen_hoger_onderwijs.csv')
df.sample(10)

,SOORT HO,PROVINCIE,BEVOEGD GEZAG NUMMER,BRIN NUMMER,INSTELLINGSNAAM,STRAATNAAM,HUISNUMMER-TOEVOEGING,POSTCODE,PLAATSNAAM,GEMEENTENUMMER,GEMEENTENAAM,DENOMINATIE,TELEFOONNUMMER,INTERNETADRES,STRAATNAAM CORRESPONDENTIEADRES,HUISNUMMER-TOEVOEGING CORRESPONDENTIEADRES,POSTCODE CORRESPONDENTIEADRES,PLAATSNAAM CORRESPONDENTIEADRES,NODAAL GEBIED CODE,NODAAL GEBIED NAAM,RPA-GEBIED CODE,RPA-GEBIED NAAM,WGR-GEBIED CODE,WGR-GEBIED NAAM,COROPGEBIED CODE,COROPGEBIED NAAM,ONDERWIJSGEBIED CODE,ONDERWIJSGEBIED NAAM,RMC-REGIO CODE,RMC-REGIO NAAM
14,hbo,Noord-Brabant,60802,02NT,Design Academy Eindhoven,Emmasingel,14,5611AZ,EINDHOVEN,772,EINDHOVEN,Algemeen bijzonder,402393939.0,www.designacademy.nl/,Postbus,2125,5600CC,EINDHOVEN,70,Eindhoven,28,Zuidoost-Brabant,37,Zuidoost-Brabant,36,Zuidoost-Noord-Brabant,22,Zuidoost-Noord-Brabant,37,Zuidoost-Brabant
11,hbo,Noord-Brabant,40205,21CW,HAS Hogeschool,Onderwijsboulevard,221,5223DE,'S-HERTOGENBOSCH,796,S HERTOGENBOSCH,Rooms-Katholiek,888903600.0,www.has.nl,Postbus,90108,5200MA,'S-HERTOGENBOSCH,69,'s-Hertogenbosch,27,Noordoost-Brabant,36,Noordoost-Brabant,35,Noordoost-Noord-Brabant,12,Noord-Noord-Brabant,36,Noord-Oost-Brabant
18,hbo,Noord-Holland,40331,28DN,Hogeschool van Amsterdam,Spui,21,1012WX,AMSTERDAM,363,AMSTERDAM,Algemeen bijzonder,205953200.0,www.hva.nl,Postbus,1025,1000BA,AMSTERDAM,44,Amsterdam,20,Zuidelijk Noord-Holland,21,Agglomeratie Amsterdam,23,Groot-Amsterdam,15,Zuid-Noord-Holland,21,Agglomeratie Amsterdam
26,hbo,Zeeland,30246,21MI,HZ University of Applied Sciences,Edisonweg,4,4382NW,VLISSINGEN,718,VLISSINGEN,Algemeen bijzonder,118489000.0,www.hz.nl,Postbus,364,4380AJ,VLISSINGEN,59,Middelburg,24,Zeeland,32,Walcheren,32,Overig Zeeland,19,Zeeland,32,Walcheren
0,hbo,Friesland,30156,31FR,NHL Stenden Hogeschool,Rengerslaan,10,8917DD,LEEUWARDEN,80,LEEUWARDEN,Overige,582512345.0,www.nhl.nl,Postbus,1080,8900CB,LEEUWARDEN,10,Leeuwarden,4,Fryslân,4,Friesland-Noord,4,Noord-Friesland,3,Friesland,4,Friesland Noord
7,hbo,Groningen,40169,25BE,Hanzehogeschool Groningen,Zernikeplein,1,9747AS,GRONINGEN,14,GRONINGEN,Algemeen bijzonder,505955610.0,www.hanze.nl/,Postbus,30030,9700RM,GRONINGEN,1,Groningen,3,Centraal-Groningen,3,Centraal en westelijk Groningen,3,Overig Groningen,1,Groningen en omstreken,3,Centraal en Westelijk Groningen
8,hbo,Limburg,41128,25JX,Zuyd Hogeschool,Nieuw-Eyckholt,300,6419DJ,HEERLEN,917,HEERLEN,Algemeen bijzonder,454006060.0,www.hszuyd.nl/,Postbus,550,6400AN,HEERLEN,79,Heerlen,33,Parkstad Limburg,41,Oostelijk Zuid-Limburg,39,Zuid-Limburg,25,Zuid-Limburg,39,Gewest Zuid-Limburg
50,wo,Utrecht,30816,23BF,Universiteit voor Humanistiek,Kromme Nieuwegracht,29,3512HD,UTRECHT,344,UTRECHT,Algemeen bijzonder,302390100.0,www.uvh.nl,Postbus,797,3500AT,UTRECHT,36,Utrecht,18,Utrecht-Midden,19,Utrecht,17,Utrecht,13,Utrecht en omstreken,19,Utrecht
39,wo,Groningen,8,21PC,Rijksuniversiteit Groningen,Broerstraat,5-7,9712CP,GRONINGEN,14,GRONINGEN,Rijks,503639111.0,www.rug.nl/,Postbus,72,9700AB,GRONINGEN,1,Groningen,3,Centraal-Groningen,3,Centraal en westelijk Groningen,3,Overig Groningen,1,Groningen en omstreken,3,Centraal en Westelijk Groningen
46,wo,Noord-Holland,75792,21PL,Vrije Universiteit Amsterdam,De Boelelaan,1105,1081HV,AMSTERDAM,363,AMSTERDAM,Algemeen bijzonder,205989898.0,www.vu.nl,De Boelelaan,1105,1081HV,AMSTERDAM,44,Amsterdam,20,Zuidelijk Noord-Holland,21,Agglomeratie Amsterdam,23,Groot-Amsterdam,15,Zuid-Noord-Holland,21,Agglomeratie Amsterdam


In [31]:
df.dtypes

SOORT HO                                       object
PROVINCIE                                      object
BEVOEGD GEZAG NUMMER                            int64
BRIN NUMMER                                    object
INSTELLINGSNAAM                                object
STRAATNAAM                                     object
HUISNUMMER-TOEVOEGING                          object
POSTCODE                                       object
PLAATSNAAM                                     object
GEMEENTENUMMER                                  int64
GEMEENTENAAM                                   object
DENOMINATIE                                    object
TELEFOONNUMMER                                float64
INTERNETADRES                                  object
STRAATNAAM CORRESPONDENTIEADRES                object
HUISNUMMER-TOEVOEGING CORRESPONDENTIEADRES      int64
POSTCODE CORRESPONDENTIEADRES                  object
PLAATSNAAM CORRESPONDENTIEADRES                object
NODAAL GEBIED CODE          

Type problems:

* (1) URLs are not recognized
* (2) Phone numbers stored as float
* (3) Codes are positive numbers (i.e. Counts)

In [32]:
typeset = visions_standard_set()

report = type_inference_report_frame(df, typeset)
print(report)

WGR-GEBIED CODE                                 visions_integer                == visions_integer                
BRIN NUMMER                                     visions_string                 == visions_string                 
PROVINCIE                                       visions_string                 == visions_string                 
GEMEENTENUMMER                                  visions_integer                == visions_integer                
STRAATNAAM CORRESPONDENTIEADRES                 visions_string                 == visions_string                 
WGR-GEBIED NAAM                                 visions_string                 == visions_string                 
NODAAL GEBIED CODE                              visions_integer                == visions_integer                
RPA-GEBIED NAAM                                 visions_string                 == visions_string                 
PLAATSNAAM                                      visions_string                 == vision

(TODO: describe how to mitigate)

In [33]:
from visions.lib.relations.integer_to_count import integer_to_count

# (3)
typeset += evolve_relation(visions_count, 'uint', integer_to_count)

# (1)
typeset += visions_url

df.loc[df['INTERNETADRES']=='NaN', 'INTERNETADRES'] = None
df.dropna(inplace=True)
df['INTERNETADRES'] = 'http://' + df['INTERNETADRES']

report = type_inference_report_frame(df, typeset)
print(report)

  SOORT HO   PROVINCIE  BEVOEGD GEZAG NUMMER BRIN NUMMER  \
0      hbo   Friesland                 30156        31FR   
1      hbo  Gelderland                 40235        25BA   
3      hbo  Gelderland                 41292        27NF   
4      hbo  Gelderland                 41344        30HD   
5      hbo  Gelderland                 41618        09OT   

                  INSTELLINGSNAAM        STRAATNAAM HUISNUMMER-TOEVOEGING  \
0          NHL Stenden Hogeschool       Rengerslaan                    10   
1     Christelijke Hogeschool Ede      Oude Kerkweg                   100   
3                           ArtEZ        Onderlangs                     9   
4  Hogeschool Van Hall Larenstein  Larensteinselaan                  26-A   
5             Iselinge Hogeschool          Bachlaan                    11   

  POSTCODE  PLAATSNAAM  GEMEENTENUMMER GEMEENTENAAM  \
0   8917DD  LEEUWARDEN              80   LEEUWARDEN   
1   6717JS     EDE GLD             228          EDE   
3   6812CE 